## Necessary Libraries
<p id=1>

Panel is an open-source Python library that lets you create custom interactive web apps and dashboards by connecting user-defined widgets to plots, images, tables, or text.

https://panel.holoviz.org


Python 3.7 - Python 3.10
- conda install -c conda-forge hvplot
- conda install -c pyviz panel

In [1]:
# Warning ignoring
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import plotly
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from plotly.subplots import make_subplots
import panel as pn
pn.extension('tabulator')
import hvplot.pandas

## Creating Dashboard


In [2]:
pop_df=pd.read_csv("structured data/eu_population.csv",encoding="UTF-8")
org_farm_df= pd.read_csv("structured data/org_area_agriprod.csv",encoding='utf-8', )

In [3]:
# Make DataFrame Pipeline Interactive
idf=org_farm_df.interactive()
pop_idf = pop_df.interactive()

In [23]:
# Define Panel widgets
year_slider = pn.widgets.IntSlider(name='Year slider', start=2012, end=2022, step=1, value=2015)
year_slider

IntSlider(end=2022, name='Year slider', start=2012, value=2015)

In [5]:
# Radio buttons for the columns related to Milk
op_yaxis = pn.widgets.RadioButtonGroup(
    name='Y axis operator', 
    options=org_farm_df.loc[:,org_farm_df.columns.str.contains("Milk")].columns.tolist(),
    button_type='success'
)

In [19]:
continents = org_farm_df['geo'].unique()

milk_pipeline = (
    idf[
        (idf.year <= year_slider) &
        (idf.geo.isin(continents))
    ]
    .groupby(['geo', 'year'])[op_yaxis].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)

In [20]:
milk_plot = milk_pipeline.hvplot(x='year', by = 'geo', y = op_yaxis, line_width=2, title='Milk')
milk_plot

In [9]:
m_yaxis = pn.widgets.RadioButtonGroup(
    name='Y axis operator', 
    options=org_farm_df.loc[:,org_farm_df.columns.str.contains("Meat")].columns.tolist(),
    button_type='success'
)

continents = org_farm_df['geo'].unique()

meat_pipeline = (
    idf[
        (idf.year <= year_slider) &
        (idf.geo.isin(continents))
    ]
    .groupby(['geo', 'year'])[m_yaxis].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='year')  
    .reset_index(drop=True)
)

In [28]:
meat_plot = meat_pipeline.hvplot(x='year', by = 'geo', y = m_yaxis, line_width=2, title='Meat')
meat_plot

## Table

In [11]:
milk_table = milk_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width')
milk_table 

In [12]:
meat_table = meat_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width')
meat_table 

## Explorer

In [13]:
hvexplorer = hvplot.explorer(org_farm_df)
hvexplorer

hvDataFrameExplorer(axes=Axes(explorer=..., height=None, legend='right', logx=False, logy=False, name='Axes08618', responsive=False, shared_axes=True, width=None, xlim=None, ylim=None), by=[], colormapping=Colormapping(clim=None, cmap='kbc_r', cnorm='linear', color=None, colorbar=None, explorer=..., name='Colormapping08646', rescale_discrete_levels=True, symmetric=False), groupby=[], kind='line', labels=Labels(clabel='', explorer=..., fontscale=1, name='Labels08667', rot=0, title='', xlabel='', ylabel=''), name='hvDataFrameExplorer08603', operations=Operations(aggregator=None, datashade=False, dynspread=False, explorer=..., name='Operations08687', rasterize=False, x_sampling=None, y_sampling=None), style=Style(alpha=1, explorer=..., name='Style08707'), x='index', y='geo', y_multi=[], z=None)

## Scatter Plot

In [14]:
scatter = idf.hvplot(by=['geo'], kind='scatter', title='Scatter', x='year', y=['area'])

## Bar chart

In [16]:
yaxis_pop = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=pop_df.iloc[:, 1:11].columns.tolist(), 
    button_type='success'
)

continents = pop_df.geo.unique()

co2_source_bar_pipeline = (
    pop_idf[
        
        (pop_idf.geo.isin(continents))
    ]
)


In [17]:
co2_source_bar_plot = co2_source_bar_pipeline.hvplot(kind='bar', 
                                                     x='geo', 
                                                     y=yaxis_pop, 
                                                     title='')
co2_source_bar_plot

## Creating Dashboard

In [31]:
#Layout using Template
template = pn.template.FastListTemplate(
    title='Dashboard', 
    sidebar=[pn.pane.Markdown("# Organic Farming Production"), 
             pn.pane.Markdown("The livestock sector is of great importance for the sustainability of rural economies and many ecosystems. It also has a high environmental impact."), 
             pn.pane.PNG('animal.png', sizing_mode='scale_both'),
             pn.pane.Markdown("## Year"),   
             year_slider],
    main=[pn.Row(pn.Column(op_yaxis, 
                           milk_plot.panel(width=700), margin=(0,25)), 
                 milk_table.panel(width=500)), 
          pn.Row(pn.Column(m_yaxis, 
              meat_plot.panel(width=700), margin=(0,25))), 
          pn.pane.Markdown("#### Create your own plot"),
          pn.Row(pn.Column(hvexplorer, margin=(0,25)))
         ],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)
# template.show()
template.servable();